In [1]:
from bs4 import BeautifulSoup
import re
import numpy as np
import os
from tqdm.notebook import tqdm

In [2]:
def clean_text(text):
    text = text.replace('!', '.')
    text = text.replace(':', ',')
    text = text.replace('--', ',')
    
    #reg = "(?<=[a-zA-Z])-(?=[a-zA-Z]{2,})"  ## comment this out please! no replacing '-'s for malay
    #r = re.compile(reg, re.DOTALL)
    #text = r.sub(' ', text)
    text = re.sub(r'\[.+?\]','',text)
    text = re.sub(r'( -| - |- )','',text)
    text = re.sub(r'(?<=[a-zA-Z])\.(?=[a-zA-Z])','. ',text)
    text = re.sub(r'(?<=[a-zA-Z])\?(?=[a-zA-Z])','? ',text)
    text = re.sub(r'(?<=[a-zA-Z])\,(?=[a-zA-Z])',', ',text)
#     text = text.replace('-', ',')
    text = text.replace(';', '.')    # replace symbols with the most relevant counterparts
    text = text.replace(' ,', ',')
    text = text.replace('♫', '')
    text = text.replace('♪ ♪',',')
    text = text.replace('♪','')
    text = text.replace('...', '')
    text = text.replace(r'.\s*\"', ',')
    text = text.replace(r',\s*"',', ')
    text = text.replace(r'"', ',')

    text = re.sub(r'--\s?--', '', text) # replace --   -- to ''
    text = re.sub(r'\s+', ' ', text)    # replace all extra whitespaces
    
    text = re.sub(r',\s?,', ',', text)  # merge commas separating only whitespace
    text = re.sub(r',\s?\.', '.', text) # , . -> ,
    text = re.sub(r'(?<=[a-zA-Z0-9]),(?=[a-zA-Z0-9])',', ',text) # say,you -> say, you
    text = re.sub(r'\?\s?\.', '?', text)# ? . -> ?
    text = re.sub(r'\s+', ' ', text)    # strip all redundant whitespace that could have been caused by preprocessing
    
    text = re.sub(r'\s+\?', '?', text)
    text = re.sub(r'\s+,', ',', text)
    text = re.sub(r'\.[\s+\.]+', '. ', text)
    text = re.sub(r'\s+\.', '.', text)
    
    return text.strip().lower()

In [9]:
dataset_dir = "./dataset/malay-dataset/local-movies-subtitle"
subtitle_text_list = []
file_list = []
for dirpath,subdirs,files in os.walk(dataset_dir):
    for f in files:
        if "ms" in f:
            file_list.append(os.path.join(dirpath,f))

for file in file_list: 
    subfile = open(file,'rb')
    lines = subfile.read()
    xml = BeautifulSoup(lines,'lxml-xml')
    subtitle_content = xml.find_all('p')
    subtitle_text = ""
    for sub in subtitle_content:
        subtitle_text = subtitle_text+" "+sub.getText()
    subtitle_text = clean_text(subtitle_text[1:])
    subtitle_text_list.append(subtitle_text)


lines = []
with open('./dataset/malay-dataset/dumping-iium.txt','r') as f:
    lines = f.readlines()
    i = 0
    prev = 0
    len_sentence = 0
    while i < len(lines):
        prev = i
        while len_sentence <= 512 and i < len(lines):
            len_sentence += len(lines[i].split())
            i+=1
        len_sentence = 0
        stri = clean_text(" ".join(lines[prev:i]))
        subtitle_text_list.append(stri)




#subtitle_text_list.extend(lines)
    

In [10]:
len(subtitle_text_list)
comma = period = question = 0
for i in tqdm(subtitle_text_list):
    for j in i:
        if j == ',':
            comma += 1
        elif j == '.':
            period += 1
        elif j == '?':
            question+=1

word_count = 0
for j in tqdm(subtitle_text_list):
    for i in j.split():
        if len(i) == 1 and i not in [',','?','.','!','>','<','(',')','[',']']:
            word_count+=1
        elif len(i) > 1:
            word_count+=1


  0%|          | 0/23642 [00:00<?, ?it/s]

  0%|          | 0/23642 [00:00<?, ?it/s]

In [11]:
from texttable import Texttable
result = Texttable()
result.add_rows([['Word count','period','comma','question'],[word_count, period, comma, question]])

print(result.draw())

+------------+---------+--------+----------+
| Word count | period  | comma  | question |
+============+=========+========+==========+
| 12264238   | 1087381 | 623625 | 55978    |
+------------+---------+--------+----------+


In [43]:

for i in range(len(subtitle_text_list)):
    subtitle_text_list[i]+='\n'
with open("dataset.txt",'w') as f:
    f.writelines(subtitle_text_list)

1117472 635745 64486
